In [83]:
from google.colab import auth
auth.authenticate_user()

import numpy as np
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open('material').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
import pandas as pd
pd.DataFrame.from_records(rows)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Atom,Udara,Kulit,Jaringan lunak,Testis,Tulang,Kandung kemih,Prostat,Urin,Isi usus,Usus,CTV,GTV
1,H,,-0.100588,-0.104472,-0.106000,-0.036000,-0.105000,-0.104000,-0.107000,-0.100000,-0.105000,-0.099000,-0.099000
2,C,,-0.228250,-0.232190,-0.100000,-0.159000,-0.096000,-0.231000,-0.003000,-0.222000,-0.113000,-0.269000,-0.269000
3,N,-0.800000,-0.046420,-0.024880,-0.021000,-0.042000,-0.026000,-0.028000,-0.010000,-0.022000,-0.026000,-0.045000,-0.045000
4,O,-0.200000,-0.619002,-0.630238,-0.764000,-0.448000,-0.761000,-0.627000,-0.873000,-0.644000,-0.750000,-0.569000,-0.569000
5,Na,,-0.000070,-0.001130,-0.002000,-0.003000,-0.002000,-0.001000,-0.004000,-0.001000,-0.001000,,
6,Mg,,-0.000060,-0.000130,,-0.002000,,,,,,,
7,P,,-0.000330,-0.001330,-0.001000,-0.094000,-0.002000,-0.002000,-0.001000,-0.002000,-0.001000,-0.018000,-0.018000
8,S,,-0.001590,-0.001990,-0.002000,-0.003000,-0.002000,-0.003000,,-0.003000,-0.001000,,
9,Cl,,-0.002670,-0.001340,-0.002000,,-0.003000,-0.002000,,-0.001000,-0.002000,,


In [90]:
rows = np.array(rows)

# Kode ini memberikan keluaran berupa nama unsur dan fraksi massanya
def mat(var):
  atom = rows[1:-2, 0]
  panjang = len(atom)             # untuk mencari banyak atom yang ada
  fraksi_massa  = rows[1:-2, var] # fraksi massa jaringan di kolom ke-var
  rho_jaringan  = rows[-3, var]   # massa jenis jaringan di kolom ke-var
  nama_jaringan = rows[0, var]    # nama jaringan di kolom ke-var
  print("$", nama_jaringan, ", massa jenis = ", rho_jaringan, "g/cm^3") # judul
  sum = 0 # digunakan untuk mengecek apakah total fraksi massa sama dengan 100 %
          # jika tidak 100 %, cek ulang fraksi massa di spreadsheet
  for i in range(panjang-1):
    if fraksi_massa[i] == "":
      pass  # apabila unsur kosong maka pass
    else:
      sum += float(fraksi_massa[i])
      print(atom[i], "\t", fraksi_massa[i])
  print("\n", sum*-100, "% benar")

mat(9)

$ Isi usus , massa jenis =  1.040 g/cm^3
H 	 -0.100000
C 	 -0.222000
N 	 -0.022000
O 	 -0.644000
Na 	 -0.001000
P 	 -0.002000
S 	 -0.003000
Cl 	 -0.001000
K 	 -0.004000
Ca 	 -0.001000

 100.0 % benar


In [98]:
# Kode ini memberikan keluaran berupa fraksi massa masing-masing unsur dalam
# jaringan setelah ditambahkan dengan Boron-10. Satuan konsentrasi Boron-10
# adalah ppm (part per milion) atau mikro gram Boron-10 per gram jaringan kanker
def fraksi_new(kons):
  kons = float(kons) / 1000000  # mengubah satuan mikro gram ke gram
  atom = rows[1:-2, 0]          # mendapatkan banyak atom
  jaringan = rows[0].shape[0]   # mendapatkan banyak jaringan
  for k in range(1, jaringan):
    fraksi_massa = rows[1:-2, k]
    panjang = len(atom)

    fraksi_massa  = rows[1:-2, k] # fraksi massa jaringan di kolom ke-k
    rho_jaringan  = rows[-3, k]   # massa jenis jaringan di kolom ke-k
    nama_jaringan = rows[0, k]    # nama jaringan di kolom ke-k

    print("\nMat[",k,"] $", nama_jaringan, ", massa jenis = ", rho_jaringan, "g/cm^3") # judul
    rasio = float(rows[-1, k])    # masing-masing jaringan berbeda
    fraksi_b10 = (kons * rasio) / (kons * rasio + 1)
    for i in range(panjang-1):
      if fraksi_massa[i] == "":
        pass        # kalau unsur kosong, maka pass
      else:
        fraksi_baru = float(fraksi_massa[i]) / (kons * rasio + 1)
        print(atom[i], "\t", fraksi_baru)
    print("10B \t",  -fraksi_b10)

fraksi_new(20)


Mat[ 1 ] $ Udara , massa jenis =  0.001 g/cm^3
N 	 -0.8
O 	 -0.2
10B 	 -0.0

Mat[ 2 ] $ Kulit , massa jenis =  1.100 g/cm^3
H 	 -0.10058749706251469
C 	 -0.22824885875570622
N 	 -0.046419767901160494
O 	 -0.618998905005475
Na 	 -6.999965000174998e-05
Mg 	 -5.9999700001499995e-05
P 	 -0.00032999835000824994
S 	 -0.0015899920500397498
Cl 	 -0.0026699866500667496
K 	 -0.0008499957500212498
Ca 	 -0.00014999925000374996
Fe 	 -9.999950000249999e-06
Zn 	 -9.999950000249999e-06
10B 	 -4.9999750001249995e-06

Mat[ 3 ] $ Jaringan lunak , massa jenis =  1.000 g/cm^3
H 	 -0.10447147764261178
C 	 -0.23218883905580473
N 	 -0.024879875600621996
O 	 -0.6302348488257559
Na 	 -0.0011299943500282497
Mg 	 -0.00012999935000324996
P 	 -0.0013299933500332498
S 	 -0.0019899900500497496
Cl 	 -0.0013399933000334998
K 	 -0.0019899900500497496
Ca 	 -0.00022999885000574998
Fe 	 -4.9999750001249995e-05
Zn 	 -2.9999850000749997e-05
10B 	 -4.9999750001249995e-06

Mat[ 4 ] $ Testis , massa jenis =  1.040 g/cm^3
H 	 -

In [97]:
# Kode ini memberikan keluaran berupa fraksi massa masing-masing unsur dalam
# jaringan sebelum ditambahkan dengan Boron-10
def fraksi_old():
  atom = rows[1:-2, 0]          # mendapatkan banyak atom
  jaringan = rows[0].shape[0]   # mendapatkan banyak jaringan
  for k in range(1, jaringan):
    fraksi_massa = rows[1:-2, k]
    panjang = len(atom)

    fraksi_massa  = rows[1:-2, k] # fraksi massa jaringan di kolom ke-k
    rho_jaringan  = rows[-3, k]   # massa jenis jaringan di kolom ke-k
    nama_jaringan = rows[0, k]    # nama jaringan di kolom ke-k

    print("\nMat[",k,"] $", nama_jaringan, ", massa jenis = ", rho_jaringan, "g/cm^3") # judul
    rasio = float(rows[-1, k])    # masing-masing jaringan berbeda
    for i in range(panjang-1):
      if fraksi_massa[i] == "":
        pass        # kalau unsur kosong, maka pass
      else:
        print(atom[i], "\t", fraksi_massa[i])

fraksi_old()


Mat[ 1 ] $ Udara , massa jenis =  0.001 g/cm^3
N 	 -0.800000
O 	 -0.200000

Mat[ 2 ] $ Kulit , massa jenis =  1.100 g/cm^3
H 	 -0.100588
C 	 -0.228250
N 	 -0.046420
O 	 -0.619002
Na 	 -0.000070
Mg 	 -0.000060
P 	 -0.000330
S 	 -0.001590
Cl 	 -0.002670
K 	 -0.000850
Ca 	 -0.000150
Fe 	 -0.000010
Zn 	 -0.000010

Mat[ 3 ] $ Jaringan lunak , massa jenis =  1.000 g/cm^3
H 	 -0.104472
C 	 -0.232190
N 	 -0.024880
O 	 -0.630238
Na 	 -0.001130
Mg 	 -0.000130
P 	 -0.001330
S 	 -0.001990
Cl 	 -0.001340
K 	 -0.001990
Ca 	 -0.000230
Fe 	 -0.000050
Zn 	 -0.000030

Mat[ 4 ] $ Testis , massa jenis =  1.040 g/cm^3
H 	 -0.106000
C 	 -0.100000
N 	 -0.021000
O 	 -0.764000
Na 	 -0.002000
P 	 -0.001000
S 	 -0.002000
Cl 	 -0.002000
K 	 -0.002000

Mat[ 5 ] $ Tulang  , massa jenis =  1.920 g/cm^3
H 	 -0.036000
C 	 -0.159000
N 	 -0.042000
O 	 -0.448000
Na 	 -0.003000
Mg 	 -0.002000
P 	 -0.094000
S 	 -0.003000
Ca 	 -0.213000

Mat[ 6 ] $ Kandung kemih , massa jenis =  1.040 g/cm^3
H 	 -0.105000
C 	 -0.096000
N 	

In [101]:
fraksi_old()


Mat[ 1 ] $ Udara , massa jenis =  0.001 g/cm^3
N 	 -0.800000
O 	 -0.200000

Mat[ 2 ] $ Kulit , massa jenis =  1.100 g/cm^3
H 	 -0.100588
C 	 -0.228250
N 	 -0.046420
O 	 -0.619002
Na 	 -0.000070
Mg 	 -0.000060
P 	 -0.000330
S 	 -0.001590
Cl 	 -0.002670
K 	 -0.000850
Ca 	 -0.000150
Fe 	 -0.000010
Zn 	 -0.000010

Mat[ 3 ] $ Jaringan lunak , massa jenis =  1.000 g/cm^3
H 	 -0.104472
C 	 -0.232190
N 	 -0.024880
O 	 -0.630238
Na 	 -0.001130
Mg 	 -0.000130
P 	 -0.001330
S 	 -0.001990
Cl 	 -0.001340
K 	 -0.001990
Ca 	 -0.000230
Fe 	 -0.000050
Zn 	 -0.000030

Mat[ 4 ] $ Testis , massa jenis =  1.040 g/cm^3
H 	 -0.106000
C 	 -0.100000
N 	 -0.021000
O 	 -0.764000
Na 	 -0.002000
P 	 -0.001000
S 	 -0.002000
Cl 	 -0.002000
K 	 -0.002000

Mat[ 5 ] $ Tulang  , massa jenis =  1.920 g/cm^3
H 	 -0.036000
C 	 -0.159000
N 	 -0.042000
O 	 -0.448000
Na 	 -0.003000
Mg 	 -0.002000
P 	 -0.094000
S 	 -0.003000
Ca 	 -0.213000

Mat[ 6 ] $ Kandung kemih , massa jenis =  1.040 g/cm^3
H 	 -0.105000
C 	 -0.096000
N 	